In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import random
import numpy as np
import socket
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from matplotlib import colors
from datetime import datetime
from pathlib import Path
from random import choices
from typing import Union, List
from futils import parse_version, snapshot
from hscpy.figures import PlotOptions
from hscpy.figures import abc as abc_fig
from hscpy.figures import sfs as sfs_fig
from hscpy import realisation, mitchell, abc, parse_path2folder_xdoty_years, variant

PATH2BIN = Path("~").expanduser() / "hsc/target/release"
assert PATH2BIN.is_dir()

In [ ]:
%%bash -s "$PATH2BIN" --out version
$1/hsc --version

In [ ]:
USE_SCRATCH = True
SAVEFIG = True
SHOW_PRIORS = True
LATEST = True

# the higher, the less precise and thus the more runs
QUANTILE, PROPORTION_RUNS_DISCARDED = 0.26, 0.2

options = PlotOptions(figsize=(7, 6), extension="svg", save=SAVEFIG)

In [ ]:
if LATEST:
    VERSION = parse_version(version)
else:
    VERSION = "v2.0.0"
PATH2SAVE = Path(f"./{VERSION}")

print("Running hsc with version:", VERSION)

if USE_SCRATCH:
    PATH2SIMS = Path("/data/scratch")
else:
    PATH2SIMS = Path("/data/home")
PATH2SIMS /= f"hfx923/hsc-draft/{VERSION}"

if socket.gethostname() == "5X9ZYD3":
    PATH2MITCHELL = Path("/mnt/c/Users/terenz01/Documents/SwitchDrive/PhD/hsc")
elif socket.gethostname() == "LAPTOP-CEKCHJ4C":
    PATH2MITCHELL = Path("/mnt/c/Users/fra_t/Documents/PhD/hsc")
else:
    PATH2MITCHELL = Path("~").expanduser()

In [ ]:
# 1. we dont use summary `Summary_cut.csv` because
#    it contains errors (i.e. duplicated and missing entries)
# 2. we dont use donor CB001 because we already have CB002 at age 0
# 3. we dont use donor KX007 because the genotype matrix is wrong,
#    they have uploded twice the same donor
donors = pd.DataFrame.from_records(
    [
        {"name": "CB002", "age": 0,  "cells": 390, "clones": 0},
        {"name": "KX001", "age": 29, "cells": 407, "clones": 0},
        {"name": "KX002", "age": 38, "cells": 380, "clones": 1},
        {"name": "SX001", "age": 48, "cells": 362, "clones": 0},
        {"name": "AX001", "age": 63, "cells": 361, "clones": 1},
        {"name": "KX008", "age": 76, "cells": 367, "clones": 12},
        {"name": "KX004", "age": 77, "cells": 451, "clones": 15},
        {"name": "KX003", "age": 81, "cells": 343, "clones": 13}, # TODO change to 328 cells
    ]
)
donors

# Load simualted data both SFS and variant fractions

## Remove runs
Remove all the runs that didn't finish running. This is required because we load runs in a numpy array with fixed size.

In [ ]:
def files2remove(years: List[int], cells: List[int]) -> List[str]:
    assert len(years) == len(cells)
    
    paths = [
        Path(f"{PATH2SIMS}/{cell}cells/sfs/{year}dot0years")
        for year, cell in zip(years, cells)
    ]
        
    files = [file.stem for path in paths for file in path.iterdir()]
    files2remove = {file for file in set(files) if files.count(file) < len(donors.age.tolist())}
    print(f"{len(files2remove)} files to remove")
    return files2remove

In [ ]:
%%time
runs2remove = files2remove(
    years=donors.age.tolist(), 
    cells=donors.cells.tolist(),
)

In [ ]:
%%time
for r in donors[["age", "cells"]].itertuples():
    for f in runs2remove:
        myf = f"{PATH2SIMS}/{r.cells}cells/sfs/{r.age}dot0years/{f}"
        try:
            Path(myf).with_suffix(".json").unlink()
        except FileNotFoundError:
            pass
        try:
            Path(myf.replace("sfs", "variant_fraction")).with_suffix(".csv").unlink()
        except FileNotFoundError:
            pass

## Load data

In [ ]:
%%time
sfs_sims = dict()
counts_sims = dict()

for r in donors[["name", "age", "cells"]].itertuples():
    print(f"\tloading sims SFS for donor {r.name} with {r.cells} cells")
    path2sfs = Path(PATH2SIMS / f"{r.cells}cells/sfs/")
    sfs_sims.update(realisation.load_all_sfs_by_age(path2sfs))
    
    print(f"\tloading sims variant counts for donor {r.name} with {r.cells} cells")
    counts_sims.update(variant.load_all_detected_var_counts_by_age(
        PATH2SIMS / f"{r.cells}cells/variant_fraction", 0.01
    ))

In [ ]:
counts = variant.variant_counts_detected_df(counts_sims)
fig, ax = plt.subplots(1, 1)
sns.lineplot(
    counts,
    x="age",
    y="variant counts detected",
    errorbar=lambda x: (np.min(x), np.max(x)),
    ax=ax,
    label="min-max",
)
sns.lineplot(
    counts,
    x="age",
    y="variant counts detected",
    errorbar="sd",
    ax=ax,
    color="orange",
    label="std",
)
ax.legend()
plt.show()
print(counts[["variant counts detected", "age"]].groupby("age").describe())

# Run ABC on the real data

## Load the data from Mitchell's paper
We have excluded two donors from the ABC:
1. exclude KX007 bc they have uploded twice the same donor
3. exclude CB001 bc it maps to to the same timepoint as CB002 (same age 0)

In [ ]:
%%time
target_sfs = {
    r.age: mitchell.sfs_donor_mitchell(r.name, r.age, PATH2MITCHELL, remove_indels=False)
    for r in donors[["name", "age"]].itertuples()
}
# TODO: rm comment
# assert [ele[2] for ele in target_sfs.values()] == donors.cells.tolist(), "cells found in genotype matrices do not match the extected nb of cells"

### Compute the summary statistics 
1. wasserstein metric
2. the number of clones
3. the KS stat (not implemented yet)

After having computed the statistics from the simulations and Mitchell's data, we won't use the data anymore but just the summary statistic dataframe `abc_mitchell`: we are going to filter (and keep) the runs by selecting from this dataframe.

In [ ]:
%%time

# 1. wasserstein
abc_mitchell = abc.sfs_summary_statistic_wasserstein(
    sfs_sims, 
    {k: ele[-1] for k, ele in target_sfs.items()}, # ele[-1] means the SFS, k is the age
    "mitchell"
)

# add information about clones from Mitchell's fig 5a
abc_mitchell = abc_mitchell.merge(
    right=counts[["age", "idx", "variant counts detected"]],
    how="left",
    left_on=["idx", "timepoint"],
    right_on=["idx", "age"],
    validate="one_to_one",
)
abc_mitchell.dropna(inplace=True)
assert (
    not abc_mitchell.isna().any().any()
), "cannot match the nb of clones data to the abc results"
abc_mitchell = donors.merge(right=abc_mitchell, how="right", on="age", validate="one_to_many")
abc_mitchell.shape

In [ ]:
%%time
# 2. number of clones
abc_mitchell["clones diff"] = (
    abc_mitchell["clones"] - abc_mitchell["variant counts detected"]
).abs()
abc_mitchell = abc.summary_statistic_relative_diff_clones(abc_mitchell)

### Show priors

In [ ]:
if SHOW_PRIORS:
    priors = abc_mitchell[["mu", "u", "s", "std", "tau"]].drop_duplicates()
    
    fig, ax = plt.subplots(1, 1, figsize=[7, 6])
    ax = abc_fig.plot_prior(priors["s"], ax=ax, binwidth=0.01)
    plt.show()

    fig, ax = plt.subplots(1, 1, figsize=[7, 6])
    ax = abc_fig.plot_prior(priors["std"], ax=ax, binwidth=0.001)
    plt.show()

    fig, ax = plt.subplots(1, 1, figsize=[7, 6])
    ax = abc_fig.plot_prior(priors["tau"], ax=ax, binwidth=0.1)
    plt.show()
    
    fig, ax = plt.subplots(1, 1, figsize=[7, 6])
    ax = abc_fig.plot_prior(priors["mu"], ax=ax, discrete=False)
    plt.show()

    fig, ax = plt.subplots(1, 1, figsize=[7, 6])
    ax = abc_fig.plot_prior(priors["u"], ax=ax)
    plt.show()

    fig, ax = plt.subplots(1, 1, figsize=[7, 6])
    sns.histplot(abc_mitchell["wasserstein"], binwidth=0.01, ax=ax)
    plt.show()

### Run abc considering all timepoints at the same time

In [ ]:
kwargs_s, kwargs_std, kwargs_mu, kwargs_tau = (
    {"binwidth": 0.01, "stat":"density"},
    {"binwidth": 0.002, "stat":"density"},
    {"discrete": True, "stat":"density"},
    {"binwidth": 0.1, "stat":"density"},
)
lims_s, lims_std, lims_mu, lims_tau = (
    abc_fig.lims(priors, "s"),
    abc_fig.lims(priors, "std"),
    abc_fig.lims(priors, "mu"),
    abc_fig.lims(priors, "tau"),
)

In [ ]:
def find_map_by_cut(
    view: pd.Series, bins: List[Union[float, int]]
) -> Union[float, None]:
    max_a_posteriori = pd.cut(view, bins=bins).mode()
    if max_a_posteriori.shape[0] > 1:
        print(
            f"cannot compute MAP because more than one mode have been found {max_a_posteriori}"
        )
        return None
    max_a_posteriori = max_a_posteriori.iloc[0]
    return max_a_posteriori.mid

In [ ]:
 for theta in zip(
            ["s", "std", "mu", "tau"],
            [lims_s, lims_std, lims_mu, lims_tau]
        ):
        print(theta)

In [ ]:
# plot all posteriors for all timepoints separately, for all metrics
for t in sorted(target_sfs.keys()):
    #for metric in ("rel clones diff", ):
    # for metric in ("clones diff", ):
    # for metric in ("wasserstein", "rel clones diff"):
    for metric in ("wasserstein", ):
        fig, axes = plt.subplots(2, 2, layout="tight", sharey=True)
        print(f"Running ABC on {metric} metric at age {t}")
        idx = abc.run_abc_per_single_timepoint(
            abc_mitchell,
            t,
            quantile=0.01,
            metric=metric,
        ).get_idx()
        print(len(idx))
        view = abc_mitchell[abc_mitchell.idx.isin(idx)]
        
        for i, (theta, lim) in enumerate(zip(
            ["s", "std", "mu", "tau"],
            [lims_s, lims_std, lims_mu, lims_tau]
        )):
            ax = axes[np.unravel_index(i, (2, 2))]
            sns.histplot(view[theta], ax=ax) #stat="percent")
            ax.set_xlim(lim)
        fig.suptitle(f"{t} years with {metric}", y=0.9, fontsize="small")
        plt.show()

In [ ]:
nb_timepoints = len(names_mitchell)
PROPORTION_RUNS_DISCARDED = 0.2
QUANTILE_SFS = 0.2
QUANTILE_CLONES = 0.8
minimum_timepoints = int(round(nb_timepoints - nb_timepoints * PROPORTION_RUNS_DISCARDED))

# run abc with different metrics per each timepoint and keep only 
# the runs that are accepted at least for minimum timepoints
print(f"Running ABC with {minimum_timepoints} minimum timepoints over {nb_timepoints}")
wasserstein_idx = set(abc.run_abc(
    abc_mitchell,
    quantile=QUANTILE_SFS,
    metric="wasserstein",
).abc_filter_on_minimum_timepoints(minimum_timepoints).idx.tolist())
print(f"ABC wasserstein kept {len(wasserstein_idx)} runs")

clones_idx = set(abc.run_abc(
    abc_mitchell,
    quantile=QUANTILE_CLONES,
    metric="rel clones diff",
).abc_filter_on_minimum_timepoints(minimum_timepoints).idx.tolist())
print(f"ABC clones kept {len(clones_idx)} runs")

runs2keep = clones_idx.intersection(wasserstein_idx)
view = abc_mitchell[abc_mitchell.idx.isin(runs2keep)]
assert not view.empty, "empty posterior"
print(f"kept {len(runs2keep)} runs")

# plots
map_mu = find_map_by_cut(view.mu, range(0, 21))
map_s = find_map_by_cut(view.s, np.arange(0, 0.45, 0.01).tolist())
map_std = find_map_by_cut(view["std"], np.arange(0, 0.15, 0.002).tolist())
"""
g1, g2, g3 = abc_fig.plot_posteriors(
    view,
    show_mean=False,
    mu_lims=abc_fig.lims(abc_mitchell, "mu"),
    s_lims=abc_fig.lims(abc_mitchell, "s"),
    std_lims=abc_fig.lims(abc_mitchell, "std"),

)
if map_mu:
    g1.ax_joint.vlines(
        map_mu,
        view.s.min(),
        view.s.max(),
        color="yellowgreen",
        label=r"MAP $\mu=$" + str(map_mu),
    )
    g2.ax_joint.vlines(
        map_mu,
        view["std"].min(),
        view["std"].max(),
        color="yellowgreen",
        label=r"MAP $\mu=$" + str(map_mu),
    )
if map_s:
    g1.ax_joint.hlines(
        map_s,
        view.mu.min(),
        view.mu.max(),
        color="yellowgreen",
        linestyle="--",
        label=r"MAP $s=$" + f" {map_s:.2f}",
    )
    g3.ax_joint.vlines(
        map_s,
        view["std"].min(),
        view["std"].max(),
        color="yellowgreen",
        label=r"MAP $s=$" + f" {map_s:.2f}",
    )

if map_std:
    g2.ax_joint.hlines(
        map_std,
        view.mu.min(),
        view.mu.max(),
        color="yellowgreen",
        linestyle="--",
        label=r"MAP $\sigma=$" + f"{map_std:.2f}",
    )
    g3.ax_joint.hlines(
        map_std,
        view.s.min(),
        view.s.max(),
        color="yellowgreen",
        linestyle="--",
        label=r"MAP $\sigma=$" + f"{map_std:.2f}",
    )

result = stats.pearsonr(view.mu, view.s)
sns.regplot(
    data=view[["mu", "s"]],
    x="mu",
    y="s",
    label=f"r={result.statistic:.2f}, p={result.pvalue:.2e}",
    scatter=False,
    line_kws={"color": "purple", "linewidth": 1},
    ax=g1.ax_joint,
)

g1.ax_joint.set_xlabel(r"$\mu$")

for g_ in {g1, g2, g3}:
    g_.ax_joint.tick_params(
        which="major",
        bottom=True,
        top=False,
        left=True,
        right=False,
        width=1.1,
        length=5,
        labelsize=14,
    )
    g_.ax_joint.tick_params(
        which="minor",
        bottom=True,
        top=False,
        left=True,
        right=False,
        width=1.1,
        length=3,
        labelsize=14,
    )
    g_.ax_marg_x.tick_params(
        which="major",
        bottom=True,
        top=False,
        left=True,
        right=False,
        width=1.1,
        length=3,
        labelsize=14,
    )
    g_.ax_marg_x.tick_params(
        which="minor",
        bottom=True,
        top=False,
        left=True,
        right=False,
        width=1.1,
        length=3,
        labelsize=14,
    )
    g_.ax_marg_y.tick_params(
        which="minor",
        bottom=True,
        top=False,
        left=True,
        right=False,
        width=1.1,
        length=3,
        labelsize=14,
    )
    g_.ax_marg_y.tick_params(
        which="major",
        bottom=True,
        top=False,
        left=True,
        right=False,
        width=1.1,
        length=3,
        labelsize=14,
    )
    g_.ax_joint.legend(fontsize=14)

if options.save:
    g1.savefig(f"posterior_mu_s.{options.extension}")
    g2.savefig(f"posterior_mu_std.{options.extension}")
    g3.savefig(f"posterior_s_std.{options.extension}")
"""
gs = []
gs.append(abc_fig.plot_results(
    view, ["s", "std"], lim1=lims_s, lim2=lims_std, kwargs1=kwargs_s, kwargs2=kwargs_std, show_mean=False
))

gs.append(abc_fig.plot_results(
    view, ["s", "mu"], lim1=lims_s, lim2=lims_mu, kwargs1=kwargs_s, kwargs2=kwargs_mu, show_mean=False
))

gs.append(abc_fig.plot_results(
    view, ["s", "tau"], lim1=lims_s, lim2=lims_tau, kwargs1=kwargs_s, kwargs2=kwargs_tau, show_mean=False
))

gs.append(abc_fig.plot_results(
    view, ["mu", "tau"], lim1=lims_mu, lim2=lims_tau, kwargs1=kwargs_mu, kwargs2=kwargs_tau, show_mean=False
))

for i, g_ in enumerate(gs):
    g_.ax_joint.tick_params(
        which="major",
        bottom=True,
        top=False,
        left=True,
        right=False,
        width=1.1,
        length=5,
        labelsize=14,
    )
    g_.ax_joint.tick_params(
        which="minor",
        bottom=True,
        top=False,
        left=True,
        right=False,
        width=1.1,
        length=3,
        labelsize=14,
    )
    g_.ax_marg_x.tick_params(
        which="major",
        bottom=True,
        top=False,
        left=True,
        right=False,
        width=1.1,
        length=3,
        labelsize=14,
    )
    g_.ax_marg_x.tick_params(
        which="minor",
        bottom=True,
        top=False,
        left=True,
        right=False,
        width=1.1,
        length=3,
        labelsize=14,
    )
    g_.ax_marg_y.tick_params(
        which="minor",
        bottom=True,
        top=False,
        left=True,
        right=False,
        width=1.1,
        length=3,
        labelsize=14,
    )
    g_.ax_marg_y.tick_params(
        which="major",
        bottom=True,
        top=False,
        left=True,
        right=False,
        width=1.1,
        length=3,
        labelsize=14,
    )
    if options.save:
        g_.fig.savefig(f"posterior{i}.{options.extension}", bbox_inches="tight", pad_inches=0)
plt.show()
verbose = False
idx2show = dict()

for t in sorted(abc_mitchell.timepoint.unique()):
    if verbose:
        title = f"age: {t} years, quantile threshold: {abc_mitchell.loc[abc_mitchell.timepoint == t, 'wasserstein'].quantile(quantile):.2f}"
    else:
        title = f"age: {round(t)} years"

    idx2show[t] = abc_fig.get_idx_smaller_distance_clones_idx(
        abc_mitchell[abc_mitchell.timepoint == t], runs2keep
    )

    fig = sfs_fig.plot_sfs_cdf(
        [idx2show[t]], target_sfs[t], sfs_sims[t], t, verbose=verbose, alpha=0.7
    )
    """fig.suptitle(
        title,
        x=0.4,
    )"""
    if options.save:
        fig.savefig(f"sfs_{t}years.{options.extension}")
    plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, layout="tight")
sns.lineplot(
    view,
    x="age",
    y="variant counts detected",
    errorbar=lambda x: (np.min(x), np.max(x)),
    ax=ax,
    label="min-max",
)
sns.lineplot(
    view,
    x="age",
    y="variant counts detected",
    errorbar="sd",
    ax=ax,
    color="orange",
    label="std",
)
sns.scatterplot(
    data=abc_mitchell[["age", "clones"]].drop_duplicates(),
    x="age",
    y="clones",
    marker="x",
    linewidths=2,
    color="purple",
    label="Mitchell",
)
ax.legend()
if options.save:
    fig.savefig(f"variants_abc.{options.extension}")
plt.show()

In [ ]:
ts = list(idx2show.keys())
mus, ss = (
    [
        abc_mitchell[(abc_mitchell.timepoint == t) & (abc_mitchell.idx == idx)].mu.iloc[
            0
        ]
        for t, idx in idx2show.items()
    ],
    [
        abc_mitchell[(abc_mitchell.timepoint == t) & (abc_mitchell.idx == idx)].s.iloc[
            0
        ]
        for t, idx in idx2show.items()
    ],
)
sns.histplot(mus, binwidth=0.5)
plt.show()
sns.histplot(ss, binwidth=0.005)
plt.show()

plt.plot(ts, mus, marker="x", mew=2)
plt.show()

plt.plot(ts, ss, marker="x", mew=2)
plt.show()

In [ ]:
assert False

## Run ABC on subsampled simulated data

### Low mu $\mu = 1.097$

In [ ]:
low_mu = (
    abc_mitchell[(abc_mitchell.mu - 1.1).abs() < 0.01]
    .drop_duplicates(subset={"mu", "s"})
    .sort_values(by=["s", "mu"])
)
idx_low_low = low_mu.iloc[1].idx
idx_low_high = low_mu.iloc[-2].idx
idx_low_medium = low_mu.iloc[int(low_mu.shape[0] / 2)].idx

#### Low mu and low s, $\mu=1.097$ and $s=0.017$

In [ ]:
view = abc_mitchell[abc_mitchell.idx == idx_low_low]
target_stem = view.path.iloc[0].stem
view

In [ ]:
%%time
_, g1, g2, g3 = abc_fig.abc_simulated_validation(
    target_stem,
    sfs_sims,
    counts,
    abc.AbcThresholds(quantile=0.4, proportion_runs_to_discard=0, nb_clones_diff=2),
    show_priors=False,
)
plt.show()

#### Low mu and medium s, $\mu=1.096$ and $s=0.189$

In [ ]:
view = abc_mitchell[abc_mitchell.idx == idx_low_medium]
target_stem = view.path.iloc[0].stem
view

In [ ]:
%%time
_, g1, g2, g3 = abc_fig.abc_simulated_validation(
    target_stem,
    sfs_sims,
    counts,
    abc.AbcThresholds(quantile=0.4, proportion_runs_to_discard=0, nb_clones_diff=2),
    show_priors=False,
)
plt.show()

#### Low mu and high s, $\mu=1.102$ and $s=0.367$

In [ ]:
view = abc_mitchell[abc_mitchell.idx == idx_low_high]
target_stem = view.path.iloc[0].stem
view

In [ ]:
%%time
_, g1, g2, g3 = abc_fig.abc_simulated_validation(
    target_stem,
    sfs_sims,
    counts,
    abc.AbcThresholds(quantile=0.4, proportion_runs_to_discard=0, nb_clones_diff=2),
    show_priors=False,
)
plt.show()

### Medium mu $\mu = 10$

In [ ]:
medium_mu = (
    abc_mitchell[(abc_mitchell.mu - 10).abs() < 0.01]
    .drop_duplicates(subset={"mu", "s"})
    .sort_values(by=["s", "mu"])
)
idx_medium_low = medium_mu.iloc[1].idx
idx_medium_high = medium_mu.iloc[-2].idx
idx_medium_medium = medium_mu.iloc[int(medium_mu.shape[0] / 2)].idx

#### Medium mu and low s, $\mu=9.991$ and $s=0.027$

In [ ]:
view = abc_mitchell[abc_mitchell.idx == idx_medium_low]
target_stem = view.path.iloc[0].stem
view

In [ ]:
%%time
_, g1, g2, g3 = abc_fig.abc_simulated_validation(
    target_stem,
    sfs_sims,
    counts,
    abc.AbcThresholds(quantile=0.4, proportion_runs_to_discard=0, nb_clones_diff=2),
    show_priors=False,
)
plt.show()

#### Medium mu and medium s, $\mu=10.004$ and $s=0.193$

In [ ]:
view = abc_mitchell[abc_mitchell.idx == idx_medium_medium]
target_stem = view.path.iloc[0].stem
view

In [ ]:
%%time
_, g1, g2, g3 = abc_fig.abc_simulated_validation(
    target_stem,
    sfs_sims,
    counts,
    abc.AbcThresholds(quantile=0.4, proportion_runs_to_discard=0, nb_clones_diff=2),
    show_priors=False,
)
plt.show()

### High mu $\mu=19.091$

In [ ]:
high_mu = (
    abc_mitchell[(abc_mitchell.mu - 19.1).abs() < 0.01]
    .drop_duplicates(subset={"mu", "s"})
    .sort_values(by=["s", "mu"])
)
idx_high_low = high_mu.iloc[1].idx
idx_high_high = high_mu.iloc[-2].idx
idx_high_medium = high_mu.iloc[int(high_mu.shape[0] / 2)].idx

#### High mu and low s, $\mu=19.091$ and $s=0.022$

In [ ]:
view = abc_mitchell[abc_mitchell.idx == idx_high_low]
target_stem = view.path.iloc[0].stem
view

In [ ]:
%%time
_, g1, g2, g3 = abc_fig.abc_simulated_validation(
    target_stem,
    sfs_sims,
    counts,
    abc.AbcThresholds(quantile=0.4, proportion_runs_to_discard=0, nb_clones_diff=2),
    show_priors=False,
)
plt.show()

#### High mu and medium s, $\mu=19.105$ and $s=0.234$

In [ ]:
view = abc_mitchell[abc_mitchell.idx == idx_high_medium]
target_stem = view.path.iloc[0].stem
view

In [ ]:
%%time
_, g1, g2, g3 = abc_fig.abc_simulated_validation(
    target_stem,
    sfs_sims,
    counts,
    abc.AbcThresholds(quantile=0.4, proportion_runs_to_discard=0, nb_clones_diff=2),
    show_priors=False,
)
plt.show()

#### HIgh mu and high s, $\mu=19.095$ and $s=0.390$

In [ ]:
view = abc_mitchell[abc_mitchell.idx == idx_high_high]
target_stem = view.path.iloc[0].stem
view

In [ ]:
%%time
_, g1, g2, g3 = abc_fig.abc_simulated_validation(
    target_stem,
    sfs_sims,
    counts,
    abc.AbcThresholds(quantile=0.4, proportion_runs_to_discard=0, nb_clones_diff=2),
    show_priors=False,
)
plt.show()